In [ ]:
!git clone https://github.com/hailiang194/CRAFT-pytorch

Cloning into 'CRAFT-pytorch'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 59 (delta 0), reused 1 (delta 0), pack-reused 56
Unpacking objects: 100% (59/59), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('CRAFT-pytorch')

In [ ]:
import torch
import time
import imgproc
from torch.autograd import Variable
import craft_utils
import numpy as np

In [ ]:
import craft

In [ ]:
from collections import OrderedDict
def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict


In [ ]:
net = craft.CRAFT()

In [ ]:
import torch.backends.cudnn as cudnn
MODEL_PATH = 'drive/MyDrive/emnist/craft_mlt_25k.pth'
if torch.cuda.is_available():
  net.load_state_dict(copyStateDict(torch.load(MODEL_PATH)))
else:
  net.load_state_dict(copyStateDict(torch.load(MODEL_PATH, map_location='cpu')))

if torch.cuda.is_available():
  net = net.cuda()
  net = torch.nn.DataParallel(net)
  cudnn.benchmark = False

In [ ]:
def test_net(net, image, text_threshold, link_threshold, low_text, cuda, poly, refine_net=None):

    # resize
    img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
    ratio_h = ratio_w = 1 / target_ratio

    # preprocessing
    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
    x = Variable(x.unsqueeze(0))                # [c, h, w] to [b, c, h, w]
    if cuda:
        x = x.cuda()

    # forward pass
    with torch.no_grad():
        y, feature = net(x)

    # make score and link map
    score_text = y[0,:,:,0].cpu().data.numpy()
    score_link = y[0,:,:,1].cpu().data.numpy()

    # refine link
    if refine_net is not None:
        with torch.no_grad():
            y_refiner = refine_net(y, feature)
        score_link = y_refiner[0,:,:,0].cpu().data.numpy()

    # Post-processing
    boxes, polys = craft_utils.getDetBoxes(score_text, score_link, text_threshold, link_threshold, low_text, poly)

    # coordinate adjustment
    boxes = craft_utils.adjustResultCoordinates(boxes, ratio_w, ratio_h)
    polys = craft_utils.adjustResultCoordinates(polys, ratio_w, ratio_h)
    for k in range(len(polys)):
        if polys[k] is None: polys[k] = boxes[k]

    # render results (optional)
    render_img = score_text.copy()
    render_img = np.hstack((render_img, score_link))
    ret_score_text = imgproc.cvt2HeatmapImg(render_img)

    return boxes, polys, ret_score_text

In [ ]:
net.eval()

DataParallel(
  (module): CRAFT(
    (basenet): vgg16_bn(
      (slice1): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (9): ReLU(inplace=True)
        (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (slice2): Sequential(


In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('drive/MyDrive/credit/credit_card_model (1).h5')

In [ ]:
def predict_digit(digit_image):
  image = cv2.resize(digit_image, (58, 85))
  predict = model.predict(image.reshape((1, 85, 58, 3)))
  # print(np.amax(predict))
  return np.argmax(predict)

In [ ]:
import json
data = None
# Opening JSON file
with open('drive/MyDrive/emnist/expected.json') as json_file:
    data = json.load(json_file)

data

{'1622431448-2020-mb-cbs-rewardsplusvisa.png': '4055123456789010',
 '800x504-CassetteCreditCard.png': '5276690000000000',
 'BMO-AIR-MILES-World-Mastercard.png': '5191231234567890',
 'BPISKYMILESPLATINUM.jpg': '5455155911234567',
 'Ceb-Credit-Card-Gold_0726.jpg': '1234567890123456',
 'Emerald_EN_Comp27_548x344_tcm341-338971.jpg': '4520123456789123',
 'Learn_Visa_ENG.png': '4537123456789012',
 'Scotia_Gold_CHIP_MC_En_MED.jpg': '5412751234567899',
 'Scotia_Magna_CHIP_MC_En.jpg': '5412751234567899',
 'aeonChinaUnicom.jpg': '8888888888888888',
 'aeonDRMAX.jpg': '8888888888888888',
 'aeonEGL.jpg': '8888888888888888',
 'aeonFerrari.jpg': '8888888888888888',
 'aeonJAL.jpg': '8888888888888888',
 'aeonJCB.jpg': '8888888888888888',
 'aeonJapanHomeCentre.jpg': '8888888888888888',
 'aeonKFC.jpg': '8888888888888888',
 'aeonOKiLike.jpg': '8888888888888888',
 'aeonSPCA.jpg': '8888888888888888',
 'aeonSowerCard.jpg': '8888888888888888',
 'aeonUnion.jpg': '8888888888888888',
 'aeonVisa.jpg': '8888888888

In [ ]:
import os

if not os.path.isdir('result'):
  os.mkdir('result')

if not os.path.isdir('digit'):
  os.mkdir('digit')

In [ ]:
def get_credit_card(image, debug=False):
    if debug: clone = image.copy()
    
    small_credit_card = False
    if image.shape[0] * image.shape[1] <= 20000:
      small_credit_card = True

    bboxes, polys, score_text = test_net(net, image, 0.4, 0.4 if small_credit_card else 99999, 0.4, torch.cuda.is_available(), False, None)
    filtered_polys = []
    for poly in polys:
      # max_locs = np.max(poly, 0)
      if poly[0, 1] >= image.shape[0] // 2 and poly[0, 1] <  2 * image.shape[0] // 3 and cv2.contourArea(poly) > 0.002 * image.shape[0] * image.shape[1]:
        filtered_polys.append(poly)
    

    formated_polys = [cv2.boundingRect(cnt) for cnt in filtered_polys]
    
    # formated_polys = sorted(formated_polys, key=lambda x: x[2] * x[3], reverse=True)[:min(len(formated_polys), 4)]
    if not small_credit_card:

        formated_polys = sorted(formated_polys, key=lambda x: np.linalg.norm([x[1]]))[:min(16, len(formated_polys))] 
        splitted_polys = []
        for poly in formated_polys:
            if poly[2] / (poly[3] / 2) > 2.25:
                splitted_polys.append([poly[0], poly[1], poly[2] // 2, poly[3]])
                splitted_polys.append([(poly[0] + poly[3]) // 2, (poly[1] + poly[2]) // 2, poly[2] // 2, poly[3]])
            else:
                splitted_polys.append(poly)

        formated_polys = splitted_polys
    formated_polys = sorted(formated_polys, key=lambda x: x[0])
    
    
    predict = ''
    area =[]
    index = 0
    for poly in formated_polys:
      # if not poly[1] < image.shape[1] * 3 / 5 and not poly[1] + poly[3] > image.shape[1] * 3 / 5:
      if not poly[1] < image.shape[0] * 3 / 5 < poly[1] + poly[3]:
        continue 
      
      # if poly[1]+ poly[3] < image.shape[0] * 3 / 5:
      #   continue

      area.append(poly[2] * poly[3])
      if debug: cv2.rectangle(clone, (poly[0], poly[1]), (poly[0] + poly[2], poly[1] + poly[3]), (225, 0, 0), 2)
      index = index + 1
      if small_credit_card:
          first_digit = image[poly[1]: poly[1] + poly[3], poly[0]: poly[0] + poly[2] // 4]
          second_digit = image[poly[1]: poly[1] + poly[3], poly[0] + poly[2] // 4: poly[0] + 2 * poly[2] // 4]
          third_digit = image[poly[1]: poly[1] + poly[3], poly[0] + 2 * poly[2] // 4: poly[0] + 3 * poly[2] // 4]
          forth_digit = image[poly[1]: poly[1] + poly[3], poly[0] + 3 * poly[2] // 4: poly[0] + poly[2]]
      
          predict = predict + '{}{}{}{}'.format(
            predict_digit(first_digit), 
            predict_digit(second_digit), 
            predict_digit(third_digit), 
            predict_digit(forth_digit))
      else:
          predict = predict + '{}'.format(predict_digit(image[poly[1]: poly[1] + poly[3], poly[0]: poly[0] + poly[2]]))

    if debug: cv2.putText(clone, predict, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (125, 0, 0), 1, cv2.LINE_AA)
    return predict, clone if debug else None

In [ ]:
import glob
import cv2
import math

total_score = 0.0
total_image = 0
for filename in glob.glob('drive/MyDrive/credit/credit/raw_card/*.*'):
    if not filename.split('/')[-1] in data.keys():
      continue

    total_image = total_image + 1
    image = cv2.imread(filename)
    predict, debug = get_credit_card(image)
    
    if len(predict) < 16:
      total_image = total_image - 1
      continue
      
    score = 0
    for i in range(len(data[filename.split('/')[-1]])):
      if i >= len(predict):
        continue
      
      if not debug is None:
        cv2.imwrite('result/{}'.format(filename.split('/')[-1]), debug)
      if predict[i] == data[filename.split('/')[-1]][i]:
          score = score + 1
    total_score = total_score + score
    print('{:50}{:<25} {:<25} {:>2} {}'.format(
        filename.split('/')[-1], predict,
        data[filename.split('/')[-1]],
        score ,
        image.shape[0] * image.shape[1]
      ))
    
    # print(['{:.4f}'.format(a / (image.shape[0] * image.shape[1])) for a in area])
print(total_score / total_image)
print(total_image)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


aeonDRMAX.jpg                                     9888989888888988          8888888888888888          12 40480
aeonFerrari.jpg                                   8888888888888888          8888888888888888          16 42380
aeonJAL.jpg                                       4448444844484444          8888888888888888           3 900000
aeonJCB.jpg                                       8388888883888888          8888888888888888          14 42380
aeonKFC.jpg                                       8888888889889874          8888888888888888          12 42380
aeonSPCA.jpg                                      8889383090888888          8888888888888888          10 42380
aeonOKiLike.jpg                                   8888888888884688          8888888888888888          14 42120
aeonVisa.jpg                                      8888888888888898          8888888888888888          15 42380
aeonWJCB.jpg                                      8888888888888888          8888888888888888          16 42380


In [ ]:
!zip result.zip result/*
!zip digit.zip digit/*

	zip warning: name not matched: result/*

zip error: Nothing to do! (result.zip)
	zip warning: name not matched: digit/*

zip error: Nothing to do! (digit.zip)
